In [1]:
import import_ipynb
import pandas as pd
import numpy as np
import sqlite3
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import decomposition
from sklearn.compose import ColumnTransformer


In [2]:
path1='./ka_devices.db'
path2='./ka_users.csv'
path3='./ka_actions.parquet'

In [3]:
def Load_and_Combine(path1,path2,path3):
    con=sqlite3.connect('./ka_devices.db')
    file1=pd.read_sql_query('SELECT * FROM devices',con)
    file2=pd.read_csv('./ka_users.csv')
    file3=pd.read_parquet(path3,engine='pyarrow')
    frame=file2.merge(file1,on='uid_s')
    frame=frame.merge(file3,on='uid_s')
    return frame

In [4]:
## filter data, we are interested in those who completed the tutorial###
def Filter_Data(Frame):
    Frame=Frame[Frame['game_stats_tutorial_complete']==1]
    nlls=Frame.isnull().any()
    s_nlls=Frame.isna().sum()
    return Frame, s_nlls


In [5]:
def Split_Col_Types(frame):
    column_types=frame.columns.to_series().groupby(frame.dtypes).groups
    types={i.name: v for i, v in column_types.items()}
    cat_types=[i for i in types['object']]
    cluster_types=[i for i in types['float64']]
    cluster_types.append('total_spend')
    return  types, cat_types, cluster_types

#types,cat_types,cluster_types=Split_Col_Types(df_F)

In [6]:
def Show_Unique_Values(Frame,cat_types):
    for col in cat_types:
        print(Frame[col].unique())
        
    

In [7]:
#Show_Unique_Values(df_F,cat_types)

In [8]:
### too many distinct values for these categories. This will be an issue when One hot Encoding. 
##### We can also use nltk and reg expression to reduce the amount of variability by grouping up.. consider this if time permits
def Count_Unique(Frame,cat_types):
    Frame[cat_types].nunique()

In [9]:
columns_drop=['game_stats_tutorial_complete','index','device_os_s','device_mapped_s','geo_s','device_manufacturer_s','device_model_s','device_gpu_s','lang_s']


In [10]:
def Drop_Cols(Frame,columns_drop):
    Frame=Frame.drop(columns_drop,axis=1)
    return Frame

In [11]:
'''


def Restruct_Data(Frame,cluster_types):
    Frame.dropna(inplace=True)
    scaler=StandardScaler()
    Frame[cluster_types]=scaler.fit(Frame[cluster_types])
    Frame[cluster_types]=scaler.transform(Frame[cluster_types])
    Frame=pd.DataFrame(Frame,columns=Frame.columns)
    Frame.set_index('uid_s',inplace=True)
    return Frame

'''
def Restruct_Data(Frame,cluster_types):
    Frame=Frame.dropna()
    names=Frame.columns
    scaler=StandardScaler()
    ct = ColumnTransformer([
        ('somename', StandardScaler(), cluster_types)
    ], remainder='passthrough')
    Frame=ct.fit_transform(Frame)
    Frame=pd.DataFrame(Frame,columns=names)
    return Frame

In [12]:
def encode_and_concat(dataframe, feature):
    dummies = pd.get_dummies(dataframe[feature])
    res = pd.concat([dataframe, dummies], axis=1)
    res = res.drop(feature, axis=1)
    return(res) 


